In [ ]:
from google.colab import drive

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
df.head()

In [ ]:
df.dropna(inplace= True)

In [ ]:
df.shape

In [ ]:
c = df.label.value_counts()*100/df.shape[0]
plt.figure(figsize = (6,6))
plt.title("Fake Vs Real News in percentages")
sns.barplot(x =c, y =c.index, orient = 'h')
plt.xlabel('News Count in percentage')
plt.ylabel('New Type')
plt.grid()
plt.show()

In [ ]:
c = df['author'].value_counts()[df.author.value_counts()> 100]
#authors with more than 100 news
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (6,6))
plt.title("Authors with more than 100 news")
sns.barplot(x =c, y =c.index, orient = 'h')
plt.xlabel('News count')
plt.ylabel('Authors')
plt.grid()
plt.show()

In [ ]:
X = df.drop('label', axis =1)
y = df['label']

In [ ]:
for i in [X,y]:
  print(i.shape)

In [ ]:
# Performing Word embedding for the titles

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

In [ ]:
import re

In [ ]:
data = X.copy()

In [ ]:
# Preprocessing the data 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in data['title']:
  c = re.sub('[^a-z,A-Z]',' ',i).lower().split()
  rev = [ps.stem(j) for j in c if j not in stopwords.words('english')]
  corpus.append(' '.join(rev))


In [ ]:
corpus[0]

In [ ]:
data['title'][0]

In [ ]:
# One hot representation
vocab_size =  10000
one_hot_represention = [one_hot(i, vocab_size) for i in corpus]

In [ ]:
# ARRIVING AT THE MAX PADDING NUMBER TO GAIN THE MAXIMUM NUMBER OF LEN FOR ALL THE SENTENCES
max_padding = pd.Series([len(i) for i in one_hot_represention]).max()+2

In [ ]:
# Pad sequences
Padding_embedded_corpus = pad_sequences(one_hot_represention, padding= 'pre', maxlen= max_padding)

In [ ]:
Padding_embedded_corpus

In [ ]:
# Creating the word embedding represation or Feature Representation and building the LSTM brained RNN
dims = 40
model = Sequential()
model.add(Embedding(vocab_size, input_length= max_padding , output_dim= dims))
model.add(LSTM(100, activation= 'elu', recurrent_activation= 'sigmoid'))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

In [ ]:
import numpy as np
X = np.array(Padding_embedded_corpus)
y = np.array(y)

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
for i in [X_train, y_train, X_test, y_test]:
  print(i.shape)

In [ ]:
# Model training
model.fit(X_train, y_train, validation_data= (X_test, y_test), epochs= 50, batch_size= 64)

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)